In [1]:
from bluepy.btle import Scanner, DefaultDelegate

class ScanDelegate(DefaultDelegate):
    def __init__(self):
        DefaultDelegate.__init__(self)

    def handleDiscovery(self, dev, isNewDev, isNewData):
        if isNewDev:
            print("Discovered device ",  dev.addr)
        elif isNewData:
            print("Received new data from", dev.addr)

scanner = Scanner().withDelegate(ScanDelegate())
devices = scanner.scan(10.0)

for dev in devices:
    print("Device %s (%s), RSSI=%d dB" % (dev.addr, dev.addrType, dev.rssi) )
    for (adtype, desc, value) in dev.getScanData():
        print("  %s = %s" % (desc, value) )

Discovered device  54:6c:0e:b1:5e:c0
Discovered device  30:fe:a2:2b:c4:f3
Discovered device  54:6c:0e:b1:5d:73
Discovered device  54:6c:0e:b1:5e:ee
Discovered device  7f:94:53:17:66:1a
Discovered device  0c:90:be:58:3b:5b
Discovered device  29:df:74:8e:83:37
Discovered device  54:6c:0e:b1:5d:57
Discovered device  66:32:02:a5:57:5d
Received new data from 54:6c:0e:b1:5d:73
Discovered device  48:28:fb:10:b4:87
Discovered device  58:2d:34:30:36:d1
Discovered device  3c:7a:ff:a8:92:aa
Discovered device  23:18:22:8e:e3:81
Discovered device  6b:dc:ed:9c:a9:16
Discovered device  fe:9a:04:e0:34:cb
Device 54:6c:0e:b1:5e:ee (public), RSSI=-62 dB
  Flags = 06
  Incomplete 16b Services = 000018aa-0000-1000-8000-00805f9b34fb
  Tx Power = 00
  0x12 = 50002003
  Complete Local Name = Ours_EEG_Band
Device 23:18:22:8e:e3:81 (random), RSSI=-85 dB
  Manufacturer = 0600010920029af11fea881b855683456e862f783015f9071bf71e9e93
Device 3c:7a:ff:a8:92:aa (random), RSSI=-78 dB
  Manufacturer = 0600010920001f95d3e1

In [3]:
from bluepy import btle
import re

class XiomiHygroThermoDelegate(object):
    def __init__(self):
        self.temperature = None
        self.humidity = None
        self.received = False

    def handleNotification(self, cHandle, data):
        print(data)
        if cHandle == 14:
            m = re.search('T=([\d\.]*)\s+?H=([\d\.]*)', ''.join(map(chr, data)))
            self.temperature = m.group(1)
            self.humidity = m.group(2)
            self.received = True

def get_data():
    address = "58:2d:34:30:36:d1"
    p = btle.Peripheral(address)
    delegate = XiomiHygroThermoDelegate()
    p.withDelegate( delegate )
    p.writeCharacteristic(0x10, bytearray([1, 0]), True)
    while not delegate.received:
        p.waitForNotifications(30.0)

    temperature = delegate.temperature
    humidity = delegate.humidity
    print("温度：",temperature,"  湿度：",humidity,"%")
get_data()

b'T=21.4 H=35.7\x00'
b'T=21.2 H=36.0\x00'
b'T=21.4 H=35.8\x00'
b'T=21.3 H=36.0\x00'
b'T=21.3 H=35.7\x00'
b'T=21.2 H=35.9\x00'
b'T=21.4 H=35.7\x00'
b'T=21.3 H=35.8\x00'
b'T=21.4 H=35.7\x00'
b'T=21.3 H=35.8\x00'
b'T=21.4 H=35.7\x00'
b'T=21.3 H=35.9\x00'


BTLEDisconnectError: Device disconnected

In [13]:
from bluepy import btle
import json
BIT0 = 1
BIT1 = 0x1 << 1
BIT2 =  0x1 << 2
BIT3 = 0x1 << 3
BIT4 = 0x1 << 4

import paho.mqtt.client as mqtt
 
HOST = "localhost"
PORT = 1883

client = mqtt.Client()
client.connect(HOST, PORT, 60)

class EEGBandDelegate(object):
    def __init__(self,address):
        self.address = address
        self.data = {"id":address}

    def handleNotification(self, cHandle, data):
        print(cHandle,data)
        flags = data[0]
        i = 1
        #fmt.Printf("HANDLE_ID_SERVICE1: % x\n",data)
        if flags & BIT0 == BIT0 :
            #Quality
            i = i + 1
        if flags & BIT1 == BIT1 :
            #Attention
            self.data['attention'] = data[i]
            i = i + 1
        if flags & BIT2 == BIT2 :
            #Meditation
            self.data['meditation'] = data[i]
            i = i + 1
        if flags & BIT3 == BIT3 :
            #BlinkStrength
            self.data['blinkStrength'] = data[i]
            i = i + 1
        print(json.dumps(self.data))
        client.publish("ble2zigbee/eegband",json.dumps(self.data),0)
connections = []
def get_data(address):
    p = btle.Peripheral(address)
    connections.append(p)
    delegate = EEGBandDelegate(address)
    p.withDelegate( delegate )
    p.writeCharacteristic(0x21, bytearray([1, 0]), True)
    try:
        while True:
            p.waitForNotifications(10.0)
    except Exception as e:
        print('Exception,',e)
    finally:
        p.disconnect()  
# get_data(address= "54:6c:0e:b1:5d:73")

In [33]:
from bluepy.btle import Scanner, DefaultDelegate

dev_list = []

class ScanDelegate(DefaultDelegate):
    def __init__(self):
        DefaultDelegate.__init__(self)

    def handleDiscovery(self, dev, isNewDev, isNewData):
        if isNewDev:
            print("Discovered device ",  dev.addr)
        elif isNewData:
            print("Received new data from", dev.addr)

scanner = Scanner().withDelegate(ScanDelegate())
devices = scanner.scan(10.0)

for dev in devices:
    print("Device %s (%s), RSSI=%d dB" % (dev.addr, dev.addrType, dev.rssi) )
    for (adtype, desc, value) in dev.getScanData():
        if desc == "Complete Local Name" and value=="Ours_EEG_Band":
            dev_list.append(dev.addr)
            print("find Ours_EEG_Band , num ",len(dev_list))

Discovered device  15:7e:1b:34:c6:2e
Discovered device  77:3b:29:c8:0b:0c
Discovered device  10:6a:44:5b:8c:8b
Discovered device  54:6c:0e:b1:5d:06
Discovered device  54:6c:0e:b1:5e:c0
Discovered device  54:6c:0e:b1:5d:5e
Discovered device  54:6c:0e:b1:5e:86
Discovered device  54:6c:0e:b1:5e:a9
Discovered device  54:6c:0e:b1:58:19
Discovered device  fe:9a:04:e0:34:cb
Received new data from 54:6c:0e:b1:58:19
Discovered device  21:e5:52:1f:7b:d4
Discovered device  54:6c:0e:b1:5d:73


KeyboardInterrupt: 

In [32]:
import time
import threading
def get_data_muilti(dev_list):
    thread_list = []
    time.clock()
    # 逐一添加线程
    for addr in dev_list:
        print(addr)
        t = threading.Thread(target=get_data, args=(addr,))
        thread_list.append(t)

    # 逐一启动线程
    for t in thread_list:
        t.start()
    try:
        while True:
            time.sleep(20)
#             break
    except Exception as e:
        print('Exception,',e)
    finally:
        for p in connections:
            p.disconnect()
get_data_muilti(dev_list)

KeyboardInterrupt: 